UC3MAL201 Machine Learning Session 10 Tutorial 10 Leon Eriksen Helgeland -
First written 05.10.2020

# Neural Network from Scratch

1. Network intialization
2. Forward propgation of inputs
3. Back propogation of errors
4. Training the Network
5. Making predictions

In [1]:
from random import random, seed, randrange
seed(42)

## Dataset pre-processing
## a. Reading the dataset
Paths to the dataset:

In [2]:
%%time
# Paths to the dataset
dataset_file = r'seeds_dataset.csv'
dataset_directory = r'C:\Users\leon.helgeland\Documents\GitHub\machine-learning-course\Session-10-neural-network-from-scratch\dataset' 
#'~/GitHub/BigData/dataset11'
path = f'{dataset_directory}\{dataset_file}'
path = 'seeds_dataset.csv'

Wall time: 0 ns


This function reads the csv file in a dictionary and converts it to a two dimensional list.

In [3]:
%%time
def read_float_csv(path):
    '''
    This function reads the csv file in a dictionary and converts it to a two dimensional list.
    '''
    with open(path,"r") as f:
        return [[float(i) for i in line.split(',')] for line in f] # Loops through lines, splits on comma, adds values to list as float(which removes '\n').

Wall time: 0 ns


Instantiating the read_csv:

In [4]:
%%time
df = read_float_csv(path)
df_org = df

Wall time: 998 µs


## b. Normalize independant values

In [5]:
# A mess that works (too late in the evening now)
normalized_value = lambda a,b,c:(a-b)/(c-b)
maxmin_ = [[max([item[j] for item in df]), min([item[j] for item in df])] for j in range(len(df[0])-1)]
l1 = [[normalized_value(item[j],maxmin_[j][1],maxmin_[j][0]) for item in df] for j in range(len(df[0])-1)]
l2 = []
for i in range(len(l1[0])):  
    row =[] 
    for item in l1: 
        row.append(item[i]) 
    l2.append(row)
#for i in range(0,210):
#    l2[i].append(df[i][7])
df = l2

## c. Hot one encode dependant column

In [6]:
%%time
# Also a mess that works (even later in the evening now, been at school for 13 hours)
column_ = []
for i in range(0,210):
    column_.append(df_org[i][7])
new_columns = []
for i in range(0,210):
    new_columns.append([])
    if column_[i] == 1:
        new_columns[i].append(1)
        new_columns[i].append(0)
        new_columns[i].append(0)
    elif column_[i] == 2:
        new_columns[i].append(0)
        new_columns[i].append(1)
        new_columns[i].append(0)
    elif column_[i] == 3:
        new_columns[i].append(0)
        new_columns[i].append(0)
        new_columns[i].append(1)
#print(new_columns)
#for i in range(0,210):
#    df[i].append(new_columns[i][0])
#    df[i].append(new_columns[i][1])
#    df[i].append(new_columns[i][2])

Wall time: 0 ns


In [7]:
%%time
X = df
y = new_columns

Wall time: 0 ns


## d. Train test split

In [8]:
%%time
def train_test_split(X_in,y_in,ratio_value_0):
    '''
    This function takes X and y as input and splits them into a training and test set depending on the ratio value. The input ratio value decides the first of the two ratio values. Ex. to get a split of 8:2 the ration value should be 8. Else 7 would become a ratio of 7:3. 
    '''
    train_X = []
    train_y = []
    X_temp = X_in
    y_temp = y_in
    train_length = int((len(X_temp)/10)*ratio_value_0)
    test_length = train_length-len(X_temp)
    counter = train_length
    #print(len(X_temp),ratio_value_0)
    while counter > 0:
        #print(counter)
        train_X.append(X_temp.pop(randrange(0,counter)))
        train_y.append(y_temp.pop(randrange(0,counter)))
        counter-=1
    test_X = X_temp
    test_y = y_temp

    return train_X, train_y, test_X, test_y

Wall time: 0 ns


In [9]:
X_train, y_train, X_test, y_test = train_test_split(X,y,8)

## 1. Network initialization
This function intializes a two layer neural network with random wights on the edges.

In [10]:
%%time
def initialize_network(n_inputs,n_hidden,n_outputs):
    '''
    Intializes a two layer neural network with random wights on the edges.
    '''
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)] # Adds random start weighting values to the layer.
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)] # +1 is the bias added to the layer (important to shift the weights in the transform function away from 0)
    network.append(output_layer)
    return network

Wall time: 0 ns


Output:
- 0: [{'weights': [0.6394267984578837, 0.025010755222666936, 0.27502931836911926]}]
- 1: [{'weights': [0.22321073814882275, 0.7364712141640124]}, {'weights': [0.6766994874229113, 0.8921795677048454]}]

Explenation:
- 0: 1 = First connection section weight from input layer to hidden layer. 2 = Second connection section weight from input layer to hidden layer. 3 = The bias added for the hidden nodes.
- 1: 1 = First connection section weight from hidden layer to output layer. 2 = The bias for the node. 3 = Second connection section weight from hidden layer to output layer. 4 = The bias for the node.

## 2. Forward propogation of inputs

In [13]:
%%time
def activate(weights, inputs):
    '''
    This function represents the internal function at work inside the artificical neuron.
    The main purpose is to sum up weights multiplied with the inputs which will then run through a transfer function to get the neuron's (local)output.
    or "Calculate neuron activation for an input"
    '''
    activation = weights[-1] # Get the current selected bias and start with that when summarizing the (local)input and weights. We use bias so we dont start with 0.
    for i in range(len(weights)-1): # Loops through all elements execept bias.
        activation += weights[i] + inputs[i] # Gives a summazion of all the inputs coming to the node. (Creates the actual important node value).
    return activation # Returns the sum of the above lines.

Wall time: 0 ns


In [14]:
%%time
from math import exp
def transfer_sigmoid(activation):
    '''
    This function transfers the neuron activation using the sigmoid function. This is an essentail operation inside the artificial neuron.
    '''
    return 1.0 / (1.0 + exp(-activation))

Wall time: 0 ns


In [15]:
%%time
def forward_propogate(network, row):
    '''
    This function forward propagates (global)input to the hidden layer an applies the activation and the transformation to create the new (local)inputs for the network output layer.
    x(input) ==> Activate : calculate(bias, weights and input values) ==> Transform : Sigmoid ==> result is input for output layer.
    '''
    inputs = row # Goes through each row in the dataframe
    for layer in network: # Loops through all layers.
        new_inputs = [] # Define a list to collect new neuron output.
        for neuron in layer: # Looops through all neurons in each layer.
            activation = activate(neuron['weights'], inputs) # Runs the activation function using inputs and weights on the neuron in selection.
            neuron['output'] = transfer_sigmoid(activation) # Then the transfer function is applied to the activation function results. The result is written to a list.
            new_inputs.append(neuron['output']) # Collects the new output-layer inputs.
        inputs = new_inputs # Writes the new outputs/inputs depend how you see it, but its towards the next layer, or in this case new (local)inputs to the output layer.
    return inputs # Returns the new output-layer inputs

Wall time: 0 ns


## 3. Back propogation of errors

In [ ]:
%%time
def transfer_derivative(output):
    '''
    This function cauclates the derivative of the input to the output in regards to the output of the output.
    '''
    return output * (1.0 - output)

In [ ]:
%%time
def backward_propagate_error(network, expected):
    '''
    This function calculates the back propagation using the difference between the actual output and the expected output. + some maths
    '''
    for i in reversed(range(len(network))):
        layer = network[i] # Backwards looping through the network layers
        errors = list() # For collecting errors
        if i != len(network)-1: # If its not the output layer
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta']) # Calculates the error for each weight or edge. Using the delta from the neuron.
                    errors.append(error)
        else: # This is the part that works with the output layer
            for j in range(len(layer)): # Loops through the amount of neurons in the layer
                neuron = layer[j] # Select neuron using the amount above
                errors.append(expected[j] - neuron['output']) # The error is the difference between the expected output and the actual output.
        for j in range(len(layer)): # Loops through the amount of neurons in the layer
            neuron = layer[j] # Select neuron using the amount above
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output']) # ehm. ?Error linked to all nodes we are considering in the layer? # How much does this node contribute as a percentage to the totalt error of the output.

## 4. Training the Network
Update the weigths (we need the learning rate)

In [ ]:
%%time
def update_weights(network, row, l_rate): # row
    '''
    This function uses the errors to update the weights in the edges between neurons to improve the output result.
    '''
    for i in range(len(network)): # Loops through the layer in the network
        inputs = row # This: [:-1] is only used when a column is target classes 
        if i != 0: # For the first layer the inputs are the network inputs. For all other layer, the input is the last layer.
            inputs = [neuron['output'] for neuron in network[i - 1]] # Selecting the input to be the output of the last layer.
        for neuron in network[i]: # looping through neurons in the active layer i
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j] # The actual weight update for each edge for each layer, learning rate multiplied by delta and input
            neuron['weights'][-1] += l_rate * neuron['delta'] # Updating the bias, since its also contributing to the output error. You dont need this if you dont have a bias. Does not have a input, becase its not a node itself, just a number that is added to the activation function.

In [ ]:
%%time
def train_network(network, train_x, train_Y, l_rate, n_epoch, n_outputs):
    '''
    This function calls all the other functions we made in a way that ensures that the whole network functions properly. It uses a fixed number of epochs. It's using online learning, because its only presented with one case at the time and based on that we update the weight. (Its also possible to do batch training giving a batch of cases at once which will be more effective. Refer to lecture for answer.)
    '''
    for epoch in range(n_epoch): # Goes through training for each specificed epoch
        sum_error = 0 # Makes a counter that sums up the total errors of all the outputs
        for i in range(len(train_X)): # For each row or event/case in the training data.
            row = train_x[i]
            outputs = forward_propogate(network, row) # Start forward propagating through the random values created by the intialization and then you are left with the output
            expected = [0 for i in range(n_outputs)] # a
            expected[train_Y] = 1 # b ##### This does not work since there is one column in the original dataset this algorithm was built upon but three columns in the new dataset i will be using.
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) # c Calculates the error based on the expected vs actual output
            backward_propagate_error(network, expected) # Backpropagation using the expected values to determine the error
            update_weights(network, row, l_rate) # Update the weights and make it ready for next element.
        print('>epoch=%d, error=%.3f' % (epoch, sum_error))
        

In [ ]:
%%time
# Train the network
n_inputs = len(X_train[0])
n_outputs = len(y_train[0])
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, )

## 5. Making predictions

In [ ]:
%%time
def predict(network, row):
    '''
    This function is made to test a prediction on a already trained network. How it does that is by just using forward propagation not backwards propagation.
    '''
    outputs = forward_propogate(network, row)
    return outputs.index(max(outputs))

# Testing

In [11]:
%%time
#seed(42)

# network = initialize_network(7,1,3)

# Test network
#network = initialize_network(2,1,2) # Input, hidden, output
#for index, layer in enumerate(network):
#    print(f'{index}: {layer}')

Wall time: 0 ns


In [ ]:
#%%time
# Intialize

#row = 

In [ ]:
#normalized_value = lambda a,b,c:(a-b)/(c-b)
#print(normalized_value(21.17,10.59,21.18))

In [ ]:
#x = 21.17
#x_min = 10.59
#x_max = 21.18
#x_new = (x - x_min) / (x_max - x_min)
#print(x_new)

In [ ]:
#%%time
#def train_test_split(y_column_index = data):
    # if y != 0 or 1:
    # then hot one encoder
#    return X_train = [], y_train = [], X_test = [], y_test = []

#def model_fit(X_train, y_train, config):